this is the file that has the final data access methods for the page

at first i had it hard coded to my own tick list so i could work on the other stuff.
i eventually coded a tick list scraper but it took too long because MP was throttling me. 

so i went back to hard coding. also i will use my friend ryan westby's tick list because he ticks literally 
every one of his climbs so there's more data. 

the very last thing i will do for this iteration is to import ryan's tick list into its own database file
so i can upload it to the free flask app hosting service, whose max file size is 100mb. currently the db
is sitting at 450mb. 

In [6]:
import sqlite3 as sl
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

con = sl.connect('my-test2.db')

# db is now indexed on row index and url

In [4]:
def getStatus(string):
    if string == '': return ''
    poss = {'Solo', 'TR', 'Follow', 'Lead', 'Lead / Onsight', 'Lead / Flash', 'Lead / Redpoint', 'Lead / Pinkpoint', 'Lead / Fell/Hung'}
    
    meta = re.split(r'\.', string)[0]
    if (meta not in poss): return ''
    return meta

def getLastPage(url):
    page = requests.get(url)
    tickSoup = BeautifulSoup(page.content, 'html.parser')
    li = tickSoup.find_all('a', class_ = 'no-click')
    noclick = [route.text for route in li]
    p = re.compile(r'\d+')
    for el in noclick:
        if p.findall(el):
            lastPage = p.findall(el)[1]
            break
    return int(lastPage)

def getPageURL(i, url):
    ext = '' if i == 1 else ('?page=%d' %i)
    url = url + ext
    return url

def getTickInfo(tick):
    url = tick['href']
    
    info = re.split(r'[·]',tick.find('i').text)
    date = info[0].strip()
    
    if len(info) > 1:
        status = getStatus(info[1].strip())
    else: 
        status = ''
        
    return (url, date, status)

def getPageTicks(url):
    pageTicks = []
    page = requests.get(url)
    tickSoup = BeautifulSoup(page.content, 'html.parser')
    userRoutes = tickSoup.find_all('a', class_ = 'route-row')
    
    for route in userRoutes:
        info = getTickInfo(route)
        pageTicks.append(info)
        
    return pageTicks

def getUserTicks(url):
    userTicks = [] 
    
    for i in range(1, getLastPage(url)+1):
        pageurl = getPageURL(i, url)
        pageTicks = getPageTicks(pageurl)
        userTicks+=pageTicks
        
    return userTicks # the ticklist is a list of tuples containing (url, date, status)

In [7]:
tickdf = pd.DataFrame(getUserTicks('https://www.mountainproject.com/user/200200801/ryan-westby/ticks'))

def getRouteInfo(url):
    with con:
        data = con.execute('select rowid, name, grade, type, area, style from routes where url = "' + url + '"')
        for row in data:
            return row
        
def insertInfo(url, row):
    tickdf.loc[tickdf['url'] == url, ['rowid','name','grade','type','area','style']] = row

def updateTickList():
    for url in tickdf['url'].tolist():
        row = getRouteInfo(url)
        insertInfo(url, row)

tickdf = tickdf.rename(columns={0:'url',1:'date',2:'status'})
        
tickdf['rowid']=0
tickdf['name']=''
tickdf['grade']=0.0
tickdf['type']=''
tickdf['area']=''
tickdf['style']=''
tickdf.set_index('url')

updateTickList()
        
# for incomplete grades backfill a middling grade
def cleanGrades(tickdf):
    tickdf.loc[tickdf['grade'] == 5.1, 'grade'] = '5.10b/c'
    tickdf.loc[tickdf['grade'] == 5.11, 'grade'] = '5.11b/c'
    tickdf.loc[tickdf['grade'] == 5.12, 'grade'] = '5.12b/c'
    return tickdf

# only care about sends... filters out boulders as byproduct because they don't start with "Lead"
def filterSends(tickdf):
    sendConditions = ['Lead / Onsight', 'Lead / Flash', 'Lead / Redpoint', 'Lead / Pinkpoint', 'Lead', 'Solo']
    tickdf = tickdf.loc[tickdf['status'].isin(sendConditions)] 
    return tickdf

tickdf = cleanGrades(tickdf)
tickdf = filterSends(tickdf)
tickdf = tickdf.replace(to_replace='None',value='Not Categorized')
tickdf.head()

,url,date,status,rowid,name,grade,type,area,style
0,https://www.mountainproject.com/route/10594967...,"Mar 20, 2021",Lead / Onsight,178206.0,Turtle Wax,5.11b,Sport,https://www.mountainproject.com/area/105732132...,Not Categorized
1,https://www.mountainproject.com/route/10589100...,"Mar 20, 2021",Lead / Redpoint,4961.0,Sunny and Steep,5.11d,Sport,https://www.mountainproject.com/area/105732132...,Not Categorized
5,https://www.mountainproject.com/route/10739515...,"Mar 20, 2021",Lead / Redpoint,128338.0,Family Circus,5.9,Sport,https://www.mountainproject.com/area/105991270...,Not Categorized
6,https://www.mountainproject.com/route/10739514...,"Mar 20, 2021",Lead / Redpoint,116379.0,Family Affair,5.8,Sport,https://www.mountainproject.com/area/105991270...,Not Categorized
8,https://www.mountainproject.com/route/10637520...,"Mar 19, 2021",Lead / Redpoint,12425.0,Mr Choad's Wild Ride,5.11b,Sport,https://www.mountainproject.com/area/105732132...,Not Categorized


ABOVE THIS COMMENT the tickdf is nice and clean with all relevant data.
--------
BELOW this comment there are functions to generate inputs for the plotting services

line chart data code

In [3]:
from datetime import datetime

# small utility function to help sort the dates in grade dictionary
def Sort_Tuple(tup):  
    tup.sort(key = lambda x: x[0])  
    return tup 

# takes the tickdf and returns a dictionary of months with corresponding PB
def monthlyPBs(df):
    monthlyPBs = {} 
    for index, row in df.iterrows(): # for every climb in the df
        month = row.monthyear
        grade = row.gradekey
        
        if month not in monthlyPBs.keys(): # if it's the first climb of the month
            monthlyPBs[month] = grade # set the high of the month to the climb grade
        else: # if it's not the first
            highgrade = monthlyPBs[month] # get the current PB of the month to compare
            if grade > highgrade: # if the current PB is higher
                monthlyPBs[month] = grade # set a new current PB 

    return monthlyPBs    

#con = sl.connect('my-test2.db')


######################## MORE TICKDF OPERATIONS TO ADD MONTHYEAR AND GRADEKEY

gradeToNum = {
    '5.0' : 0, 'easy' : 0, 'Easy' : 0, '5.1' : 1, '5.2' : 2, '5.3' :  3, '5.4' : 4,
    '5.5' : 5, '5.6' : 6, '5.7-' : 7, '5.7' : 7, '5.7+' : 7, '5.8-' : 8, '5.8' : 8, 
    '5.8+' : 8, '5.9-' : 9, '5.9' : 9, '5.9+' : 9,'5.10a' : 10, '5.10-' : 10, 
    '5.10a/b' : 10, '5.10b' : 11, '5.10b/c' : 12, '5.10' : 12, '5.10c' : 13, 
    '5.10d' : 14, '5.10+' : 14, '5.10c/d' : 14, '5.11a' : 15, '5.11-' : 15, '5.11a/b' : 15,
    '5.11b' : 16,'5.11b/c' : 17, '5.11' : 17, '5.11c' : 18, '5.11d' : 19, '5.11+' : 19, 
    '5.11c/d' : 19, '5.12a' : 20, '5.12-' : 20, '5.12a/b' : 20, '5.12b' : 21, '5.12b/c' : 22, 
    '5.12' : 22, '5.12c' : 23, '5.12d' : 24, '5.12+' : 24, '5.12c/d' : 24, '5.13a' : 25, 
    '5.13-' : 25, '5.13a/b': 25, '5.13b': 26, '5.13b/c': 27, '5.13': 27, '5.13c': 28, 
    '5.13d': 29, '5.13+': 29, '5.13c/d': 29, '5.14a': 30, '5.14-': 30, '5.14a/b': 30, '5.14b': 31,
    '5.14b/c': 32, '5.14': 32, '5.14c': 33, '5.14d': 34, '5.14+': 34, '5.14c/d': 34,
    '5.15a': 35, '5.15-': 35, '5.15a/b': 35, '5.15b': 36, '5.15b/c': 37, '5.15': 37, '5.15c': 38,
    '5.15d': 39, '5.15+': 39, '5.15c/d': 39, '5.16a': 40, '5.16-': 40, '5.16a/b': 40, '5.16b': 41,
    '5.16b/c': 41, '5.16': 41, '5.16c': 42
}


monthyear = []
gradekey = []

# take in a date string from tickdf and append a formatted date to monthyear list
def getMonthYear(datestring):
    string = datetime.strptime(datestring, '%b %d, %Y')
    month = string.strftime('%Y-%m')
    monthyear.append(month) # add monthyear value

# because you climbing grades are unsortable as numbers OR strings. have to translate them to another scale
def translateGrade(gradestring):
    if gradestring in gradeToNum.keys(): # also, for every tick, if the grade is in the translator
        grade = gradeToNum[gradestring] # insert the new gradekey in the column
        gradekey.append(grade)
    else:
        grade = 0 # if it's not in the translator it's probably an iceclimb or something, skip it
        gradekey.append(grade)

for index, row in tickdf.iterrows(): # for every tick in the ticklist
    datestring = row.date
    getMonthYear(datestring)

    gradestring = str(row.grade)
    translateGrade(gradestring)
        
tickdf['monthyear'] = monthyear
tickdf['gradekey'] = gradekey

monthlyPBs = monthlyPBs(tickdf) # this is gradeDict
highlist = list(monthlyPBs.items()) # highlist isn't used anywhere else, it's just an intermediate to getting a datefill list
highlist = Sort_Tuple(highlist)
datefill = pd.date_range(highlist[0][0],highlist[-1][0], freq='MS').strftime("%Y-%m").tolist()
# datefill contains a backfilled list of months from earliest tick to last tick. need it for making datapoints below




# takes a grade dictionary and returns a list of most recent PBs
def recentHighs(gradeDict):
    tickedMonths = list(gradeDict.keys())
    list.sort(tickedMonths)
    
    lastPB = 0.0
    recentGradeList = []
    
    for month in datefill: # for month in datefill
        if month in tickedMonths: # check if there is a tick logged for the month
            recentGradeList.append(gradeDict[month]) # if it is, get the grade 
            lastPB = gradeDict[month] # and set the most recent PB to the grade
        else:
            recentGradeList.append(lastPB) # if no tick logged for the month, just add the previous month's
    
    return recentGradeList

# takes a grade dictionary and returns a list of personal bests (PBs)
def maxGrades(gradeDict):
    highGradeList = []
    tickedMonths = list(gradeDict.keys())
    list.sort(tickedMonths)
    
    highGrade = 0.0
    for month in datefill: # for every month
        if month in tickedMonths: # check if there is a climbing grade in the ticked maxes
            grade = gradeDict[month]
            if grade > highGrade: # if that month's grade is a PBest
                highGrade = grade
                highGradeList.append(highGrade) # add it to the list
            else:
                highGradeList.append(highGrade) # otherwise add the old PBest
        else:
            highGradeList.append(highGrade) # if no climbs in that month, append old PBest
    return highGradeList

pie chart code:

In [56]:
def getMissingCounts(li):
    allstyles = set(['Face','Overhang','Crack','None'])
    set1 = set(li)
    return list(sorted(allstyles - set2))

def backfillCounts(incompleteDict, missingli):
    for label in missingli:
        incompleteDict[label] = 0
    return incompleteDict

def regularizeCounts(df):
    incompleteDict = df['style'].value_counts().to_dict()
    missingLi = getMissingCounts(incompleteDict.keys())
    backfillCounts(incompleteDict,missingLi)
    return incompleteDict

def labelsAndCounts(di):
    labels = sorted(list(di.keys()))
    counts = [di[ele] for ele in labels]
    return labels, counts

ten = tickdf[['grade','style']][tickdf.grade.str.match(r'(^5.10.*)')==True]
ele = tickdf[['grade','style']][tickdf.grade.str.match(r'(^5.11.*)')==True]
twelve = tickdf[['grade','style']][tickdf.grade.str.match(r'(^5.12.*)')==True]

tendict = regularizeCounts(ten)
eledict = regularizeCounts(ele)
twelvedict = regularizeCounts(twelve)
alldict = regularizeCounts(tickdf[['grade','style']])

tenlabels, tencounts = labelsAndCounts(tendict)
elelabels, elecounts = labelsAndCounts(eledict)
twelvelabels, twelvecounts = labelsAndCounts(twelvedict)
alllabels, allcounts = labelsAndCounts(alldict)

In [62]:
tickdf = tickdf[['url','date','status','name','grade','type','style']]

In [11]:
import sqlite3 as sl
con = sl.connect('my-test.db')

In [5]:
tickdf.to_sql('ryan_westby',con, if_exists='replace')

In [1]:
import pandas as pd
import sqlite3 as sl
con = sl.connect('my-test.db')

tickdf = pd.read_sql('select * from ryan_westby', con, index_col='url')

In [4]:
tickdf.head()

,index,date,status,rowid,name,grade,type,area,style,monthyear,gradekey
url,,,,,,,,,,,
https://www.mountainproject.com/route/105949674/turtle-wax,0,"Mar 20, 2021",Lead / Onsight,178206.0,Turtle Wax,5.11b,Sport,https://www.mountainproject.com/area/105732132...,Not Categorized,2021-03,16
https://www.mountainproject.com/route/105891002/sunny-and-steep,1,"Mar 20, 2021",Lead / Redpoint,4961.0,Sunny and Steep,5.11d,Sport,https://www.mountainproject.com/area/105732132...,Not Categorized,2021-03,19
https://www.mountainproject.com/route/107395153/family-circus,5,"Mar 20, 2021",Lead / Redpoint,128338.0,Family Circus,5.9,Sport,https://www.mountainproject.com/area/105991270...,Not Categorized,2021-03,9
https://www.mountainproject.com/route/107395148/family-affair,6,"Mar 20, 2021",Lead / Redpoint,116379.0,Family Affair,5.8,Sport,https://www.mountainproject.com/area/105991270...,Not Categorized,2021-03,8
https://www.mountainproject.com/route/106375205/mr-choads-wild-ride,8,"Mar 19, 2021",Lead / Redpoint,12425.0,Mr Choad's Wild Ride,5.11b,Sport,https://www.mountainproject.com/area/105732132...,Not Categorized,2021-03,16


In [11]:
con.execute('drop table ryan_westby')

In [7]:
data = con.execute('select * from ryan_westby')
for row in data:
    print(row)

('https://www.mountainproject.com/route/105949674/turtle-wax', 0, 'Mar 20, 2021', 'Lead / Onsight', 178206.0, 'Turtle Wax', '5.11b', 'Sport', 'https://www.mountainproject.com/area/105732132/sunny-and-steep', 'Not Categorized', '2021-03', 16)
('https://www.mountainproject.com/route/105891002/sunny-and-steep', 1, 'Mar 20, 2021', 'Lead / Redpoint', 4961.0, 'Sunny and Steep', '5.11d', 'Sport', 'https://www.mountainproject.com/area/105732132/sunny-and-steep', 'Not Categorized', '2021-03', 19)
('https://www.mountainproject.com/route/107395153/family-circus', 5, 'Mar 20, 2021', 'Lead / Redpoint', 128338.0, 'Family Circus', '5.9', 'Sport', 'https://www.mountainproject.com/area/105991270/conundrum-crag', 'Not Categorized', '2021-03', 9)
('https://www.mountainproject.com/route/107395148/family-affair', 6, 'Mar 20, 2021', 'Lead / Redpoint', 116379.0, 'Family Affair', '5.8', 'Sport', 'https://www.mountainproject.com/area/105991270/conundrum-crag', 'Not Categorized', '2021-03', 8)
('https://www.mou